In [63]:
#import all essential libraries
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.metrics import mean_squared_error,r2_score
## Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout

In [64]:
df=pd.read_excel('Book01.xlsx')
df.head()

,Date,MB033,MB034,MB035,MB036
0,2021-01-01,site off,4,71,40
1,2021-01-02,88,66,52,12
2,2021-01-03,52,40,54,72
3,2021-01-04,49,34,32,28
4,2021-01-05,82,32,85,49


In [65]:
df.tail()

,Date,MB033,MB034,MB035,MB036
498,2022-05-14,98,50,19,60
499,2022-05-15,3,0,66,48
500,2022-05-16,75,54,37,10
501,2022-05-17,43,43,20,37
502,2022-05-18,7,56,74,42


In [66]:
#check for missing values in train data
df.isnull().sum()
 #No missing valuues



Date     0
MB033    0
MB034    0
MB035    0
MB036    0
dtype: int64

Here for better understanding of the data, We can eloborate as month and weekday wise.

In [67]:
df['MB033']

0      site off
1            88
2            52
3            49
4            82
         ...   
498          98
499           3
500          75
501          43
502           7
Name: MB033, Length: 503, dtype: object

In [68]:
df['MB033'] = df['MB033'].replace(['site off'],0)
df['MB034'] = df['MB034'].replace(['site off'],0)
df['MB035'] = df['MB035'].replace(['site off'],0)
df['MB036'] = df['MB036'].replace(['site off'],0)









In [69]:
arr1 = df['MB033'].values
arr1

array([  0,  88,  52,  49,  82,   0,  62,  77,  60,  37,  58,  64,  88,
        49,  38,  87,  22,  40,  85,  54,  87,  10,  60,  23,  66,  67,
        11,  76,  40,  68,  63,   0,  72,  67,  42,  85,  11,  22,  91,
        68,  19,  74,  45,   7,  57,   0,  54,  24,  80,  52,  51,  18,
         7,  72,  85,  77,  35,  93,  64,   0,  70,  97,   6,  42,  52,
        98,  11,  36,  82,  28,  84,   2,  68,  67,  54,   5,  82,  95,
        42,  72,  95,  94,  15,  42,  46,  95,  26,  91,  67,  31,   0,
        51,  30,  24,  72,  13, 100,  36,  69,  76,   5,  52,  71,  62,
        60,  57,  41,  39,  63,  63,  38,  93,   2,  39,  93,  44,  77,
        97,  54,  14,   0,  78,  27,  65,   9,   8,  36,  26,  54,  30,
        87,  39,  49,  20,  41,  33,  70,  89,   8,  76,  14,  63,  41,
        58,  61,  56,  20,  88,  58,  71,  55,   0,  88,  45,  39,  29,
        97,   1,  25,  23,  15,  47,  15,  47,  75,  21,  39,  30,  65,
        33,  94,   9,  70,  74,  72,  19,  38,  40,  71,  27,  1

In [70]:
import numpy as np
def splitSequence(seq, n_steps):
    
    #Declare X and y as empty list
    X = []
    y = []
    
    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps
        
        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break
            
        #Create input and output sequence
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]
        
        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        pass
    #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)
    
    return X,y 
    
    pass

n_steps = 10
X, y = splitSequence(arr1, n_steps = 10)

In [71]:
X.shape,y.shape

((493, 10), (493,))

In [72]:
for i in range(len(X)):
    print(X[i], y[i])

[ 0 88 52 49 82  0 62 77 60 37] 58
[88 52 49 82  0 62 77 60 37 58] 64
[52 49 82  0 62 77 60 37 58 64] 88
[49 82  0 62 77 60 37 58 64 88] 49
[82  0 62 77 60 37 58 64 88 49] 38
[ 0 62 77 60 37 58 64 88 49 38] 87
[62 77 60 37 58 64 88 49 38 87] 22
[77 60 37 58 64 88 49 38 87 22] 40
[60 37 58 64 88 49 38 87 22 40] 85
[37 58 64 88 49 38 87 22 40 85] 54
[58 64 88 49 38 87 22 40 85 54] 87
[64 88 49 38 87 22 40 85 54 87] 10
[88 49 38 87 22 40 85 54 87 10] 60
[49 38 87 22 40 85 54 87 10 60] 23
[38 87 22 40 85 54 87 10 60 23] 66
[87 22 40 85 54 87 10 60 23 66] 67
[22 40 85 54 87 10 60 23 66 67] 11
[40 85 54 87 10 60 23 66 67 11] 76
[85 54 87 10 60 23 66 67 11 76] 40
[54 87 10 60 23 66 67 11 76 40] 68
[87 10 60 23 66 67 11 76 40 68] 63
[10 60 23 66 67 11 76 40 68 63] 0
[60 23 66 67 11 76 40 68 63  0] 72
[23 66 67 11 76 40 68 63  0 72] 67
[66 67 11 76 40 68 63  0 72 67] 42
[67 11 76 40 68 63  0 72 67 42] 85
[11 76 40 68 63  0 72 67 42 85] 11
[76 40 68 63  0 72 67 42 85 11] 22
[40 68 63  0 72 67 42

In [73]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X[:2])

[[[ 0]
  [88]
  [52]
  [49]
  [82]
  [ 0]
  [62]
  [77]
  [60]
  [37]]

 [[88]
  [52]
  [49]
  [82]
  [ 0]
  [62]
  [77]
  [60]
  [37]
  [58]]]


In [74]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [75]:
model = tf.keras.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(layers.Dense(1))

In [76]:
model.layers

In [77]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [79]:
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
16/16 [==============================] - 6s 18ms/step - loss: 1374.9054 - accuracy: 0.0081
Epoch 2/200
16/16 [==============================] - 1s 34ms/step - loss: 941.3866 - accuracy: 0.0061
Epoch 3/200
16/16 [==============================] - 0s 21ms/step - loss: 908.2450 - accuracy: 0.0061
Epoch 4/200
16/16 [==============================] - 0s 18ms/step - loss: 912.1852 - accuracy: 0.0061
Epoch 5/200
16/16 [==============================] - 0s 22ms/step - loss: 901.1199 - accuracy: 0.0061
Epoch 6/200
16/16 [==============================] - 0s 28ms/step - loss: 2372.1423 - accuracy: 0.0061
Epoch 7/200
16/16 [==============================] - 0s 18ms/step - loss: 947.6139 - accuracy: 0.0061
Epoch 8/200
16/16 [==============================] - 0s 18ms/step - loss: 905.3208 - accuracy: 0.0061
Epoch 9/200
16/16 [==============================] - 0s 16ms/step - loss: 882.8939 - accuracy: 0.0061
Epoch 10/200
16/16 [==============================] - 0s 19ms/step - loss: 910.8

In [80]:
test_data = arr1[-10:]
test_data = test_data.reshape((1, n_steps, n_features))
test_data

array([[[24],
        [ 1],
        [70],
        [92],
        [84],
        [98],
        [ 3],
        [75],
        [43],
        [ 7]]])

In [81]:
predictNextNumber = model.predict(test_data, verbose=1)
print(predictNextNumber)

1/1 [==============================] - 0s 489ms/step
[[43.633305]]
